# Reading Comprehension

In [1]:
!nvidia-smi
!lscpu

Tue Jul  6 08:03:53 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   69C    P8    11W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install hazm==0.7.0
!pip install seqeval==1.2.2
!pip install sentencepiece==0.1.96
!pip install transformers==4.7.0
!pip install clean-text[gpl]==0.4.0
!pip install sacrebleu==1.5.1

     |████████████████████████████████| 317kB 8.4MB/s 
     |████████████████████████████████| 1.4MB 36.8MB/s 
     |████████████████████████████████| 235kB 50.3MB/s 
  Created wheel for nltk: filename=nltk-3.3-cp37-none-any.whl size=1394488 sha256=b44a881de05d2da9cacc53ce1d35e5302e7994ef2e2079df8a39deec8b1fabf3
  Stored in directory: /root/.cache/pip/wheels/d1/ab/40/3bceea46922767e42986aef7606a600538ca80de6062dc266c
  Created wheel for libwapiti: filename=libwapiti-0.2.1-cp37-cp37m-linux_x86_64.whl size=154153 sha256=eaf6c7e9a8c2e372835fd97f472e67c7c41c672cf62af7af8d7db5be55aee8cd
  Stored in directory: /root/.cache/pip/wheels/66/15/54/4510dce8bb958b1cdd2c47425cbd1e1eecc0480ac9bb1fb9ab
Successfully built nltk libwapiti
  Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5
     |████████████████████████████████| 51kB 4.6MB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-cp37-none-any.whl size=16184 sha256=2ee9cafa7f0287e

In [3]:
!pip install PyDrive
import os
import IPython.display as ipd
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [4]:
# Import required packages
import os
import gc
import re
import hazm
import time
import json
import sacrebleu
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader

import transformers
from transformers import AutoConfig, AutoTokenizer
from transformers import AutoModelForSequenceClassification
from transformers import MT5Config, MT5ForConditionalGeneration, MT5Tokenizer
from transformers.data.processors.squad import SquadV2Processor
from transformers.data.metrics.squad_metrics import normalize_answer, compute_exact, compute_f1, merge_eval, \
    make_eval_dict, apply_no_ans_threshold, find_all_best_thresh, squad_evaluate

from cleantext import clean

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

print()
print('numpy', np.__version__)
print('pandas', pd.__version__)
print('transformers', transformers.__version__)
print('torch', torch.__version__)
print()

# If there's a GPU available...
if torch.cuda.is_available():    
    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")



numpy 1.19.5
pandas 1.1.5
transformers 4.7.0
torch 1.9.0+cu102

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [5]:
class ReadingComprehensionDataset(torch.utils.data.Dataset):
    """ Create a PyTorch dataset for Reading Comprehension. """

    def __init__(self, input_data, tokenizer, max_length):
        self.input_data = input_data
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.input_data)

    def __getitem__(self, item):
        pair = self.input_data[item]['context'] + "\n" + self.input_data[item]['question']
        encoding = self.tokenizer(
            pair,
            add_special_tokens=True,
            max_length=self.max_length,
            truncation=True,
            padding='max_length',
            return_tensors="pt"
        )
        inputs = {
            'qas_id': self.input_data[item]['qas_id'],
            'context_question': pair,
            'context': self.input_data[item]['context'],
            'question': self.input_data[item]['question'],
            'input_ids': encoding.input_ids.flatten(),
            'attention_mask': encoding.attention_mask.flatten()
        }
        return inputs


class ReadingComprehension:
    def __init__(self, model_name, model_type):
        self.normalizer = hazm.Normalizer()
        self.model_name = model_name
        if model_type.lower() == "mt5":
            self.tokenizer = MT5Tokenizer.from_pretrained(model_name)
            self.model = MT5ForConditionalGeneration.from_pretrained(model_name)
            self.config = MT5Config.from_pretrained(self.model_name)
        else:
            self.tokenizer = AutoTokenizer.from_pretrained(self.model_name)
            self.model = AutoModelForSequenceClassification.from_pretrained(self.model_name)
            self.config = AutoConfig.from_pretrained(self.model_name)
            self.id2label = self.config.id2label
            self.label2id = self.config.label2id

    def load_dataset_test_file(self, dataset_name, dataset_file, **kwargs):
        if dataset_name.lower() == "parsinlu":
            if not os.path.exists(dataset_file):
                print(f'{dataset_file} not exists!')
                return
            processor = SquadV2Processor()
            examples = processor.get_dev_examples(dataset_file[:dataset_file.rfind('/')],
                                                  filename=dataset_file[dataset_file.rfind('/') + 1:])
            return examples

    def mt5_reading_comprehension_inference(self, context, question, device):
        if not self.model or not self.tokenizer:
            print('Something wrong has been happened!')
            return

        new_input = []
        for q1, q2 in zip(context, question):
            new_input.append(q1 + "\n" + q2)

        tokenized_batch = self.tokenizer(
            new_input,
            padding=True,
            return_tensors="pt"
        )

        gc.collect()
        torch.cuda.empty_cache()
        # Tell pytorch to run this model on the GPU.
        if device.type != 'cpu':
            self.model.cuda()

        input_ids = tokenized_batch.input_ids.to(device)
        attention_mask = tokenized_batch.attention_mask.to(device)
        outputs = self.model.generate(input_ids=input_ids,
                                      attention_mask=attention_mask)
        predictions = self.tokenizer.batch_decode(outputs, skip_special_tokens=True)
        return predictions

    def mt5_evaluation(self, examples, device, max_length, batch_size=4):
        if not self.model or not self.tokenizer:
            print('Something wrong has been happened!')
            return

        input_data = []
        for example in examples:
            gold_answers = [answer["text"] for answer in example.answers if normalize_answer(answer["text"])]
            if not gold_answers:
                # For unanswerable questions, only correct answer is empty string
                gold_answers = [""]
            input_data.append({
                "qas_id": example.qas_id,
                "context": example.context_text,
                "question": example.question_text,
                "answers": example.answers,
                "gold_answers": gold_answers
            })

        qas_answers = {sample['qas_id']: sample['gold_answers'] for sample in input_data}
        dataset = ReadingComprehensionDataset(input_data=input_data, tokenizer=self.tokenizer, max_length=max_length)
        data_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size)
        print(f'#input_data:{len(input_data)}')
        print("#batch:", len(data_loader))

        gc.collect()
        torch.cuda.empty_cache()
        # Tell pytorch to run this model on the GPU.
        if device.type != 'cpu':
            self.model.cuda()

        total_time = 0
        output_predictions = []
        exact_scores, f1_scores, no_answer_probs, preds = {}, {}, {}, {}
        print("Start to evaluate test data ...")
        for step, batch in enumerate(data_loader):
            b_qas_id = batch['qas_id']
            b_context_question = batch['context_question']
            b_input_ids = batch['input_ids']
            b_attention_mask = batch['attention_mask']

            # move tensors to GPU if CUDA is available
            b_input_ids = b_input_ids.to(device)
            b_attention_mask = b_attention_mask.to(device)

            # This will return the loss (rather than the model output) because we have provided the `labels`.
            with torch.no_grad():
                start = time.monotonic()
                b_outputs = self.model.generate(input_ids=b_input_ids, attention_mask=b_attention_mask)
                end = time.monotonic()
                total_time += end - start
                print(f'inference time for step {step}: {end - start}')

            b_predictions = self.tokenizer.batch_decode(b_outputs, skip_special_tokens=True)
            for i in range(len(b_predictions)):
                qas_id = b_qas_id[i]
                gold_answers = qas_answers[qas_id]
                predicted_answer = b_predictions[i]
                exact_scores[qas_id] = max(compute_exact(a, predicted_answer) for a in gold_answers)
                f1_scores[qas_id] = max(compute_f1(a, predicted_answer) for a in gold_answers)

                output_predictions.append((
                    qas_id,
                    batch['context'][i],
                    batch['question'][i],
                    gold_answers,
                    predicted_answer,
                    exact_scores[qas_id],
                    f1_scores[qas_id]
                ))
                no_answer_probs[qas_id] = 0.0
                preds[qas_id] = predicted_answer

        print("total inference time:", total_time)
        print("total inference time / #samples:", total_time / len(input_data))

        # evaluate
        evaluation_results = squad_evaluate(examples, preds)
        print("evaluation results:\n", evaluation_results)
        return evaluation_results, output_predictions


In [6]:
model_name='persiannlp/mt5-base-parsinlu-squad-reading-comprehension'
rc_model = ReadingComprehension(model_name=model_name, model_type="mt5")
print(rc_model.config)


MT5Config {
  "_name_or_path": "/home/patrick/hugging_face/t5/mt5-base",
  "architectures": [
    "MT5ForConditionalGeneration"
  ],
  "d_ff": 2048,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "gated-gelu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "layer_norm_epsilon": 1e-06,
  "model_type": "mt5",
  "num_decoder_layers": 12,
  "num_heads": 12,
  "num_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_num_buckets": 32,
  "tie_word_embeddings": false,
  "tokenizer_class": "T5Tokenizer",
  "transformers_version": "4.7.0",
  "use_cache": true,
  "vocab_size": 250112
}



## Sample Inference

In [7]:
context_list = [
    "یک شی را دارای تقارن می‌نامیم زمانی که ان شی را بتوان به دو یا چند قسمت تقسیم کرد که آن‌ها قسمتی از یک طرح سازمان یافته باشند یعنی بر روی شکل تنها جابجایی و چرخش و بازتاب و تجانس انجام شود و در اصل شکل تغییری به وجود نیایید آنگاه ان را تقارن می‌نامیم مرکز تقارن:اگر در یک شکل نقطه‌ای مانندA وجود داشته باشد که هر نقطهٔ روی شکل (محیط) نسبت به نقطه یAمتقارن یک نقطهٔ دیگر شکل (محیط) باشد، نقطهٔ Aمرکز تقارن است. یعنی هر نقطه روی شکل باید متقارنی داشته باشد شکل‌های که منتظم هستند و زوج ضلع دارند دارای مرکز تقارند ولی شکل‌های فرد ضلعی منتظم مرکز تقارن ندارند. متوازی‌الأضلاع و دایره یک مرکز تقارن دارند ممکن است یک شکل خط تقارن نداشته باشد ولی مرکز تقارن داشته باشد. (منبع:س. گ)",
    "شُتُر یا اُشتر را که در زبان پهلوی (ushtar)[نیازمند منبع] می‌گفتند حیوانی است نیرومند و تنومند با توش و توان بالا از خانواده شتران؛ شبه نشخوارکننده و با دست و گردنی دراز. بر پشت خود یک یا دو کوهان دارد که ساختارش از پیه و چربی است. در دین اسلام گوشت او حلال است. اما ذبح آن با دیگر جانوران حلال گوشت متفاوت است و آن را نحر (بریدن گلو) می‌کنند و اگر سر آن را مانند گوسفند پیش از نحر ببرند گوشت آن حلال نیست. شیرش نیز نوشیده می‌شود ولی بیشتر کاربرد بارکشی دارد. پشم و پوستش نیز برای ریسندگی و پارچه‌بافی و کفش‌دوزی کاربرد دارد.  گونه‌های دیگری از شتران نیز در آمریکای جنوبی زندگی می‌کنند، به نام‌های لاما، آلپاکا، گواناکو که دارای کوهان نیستند.  شتر ویژگی‌های خاصّی دارد که مهم‌ترین آن‌ها تحمّل شرایط سخت صحرا و دماهای گوناگون و به‌ویژه گرمای شدید تابستان و کمبود آب و علوفه است. ترکیب جسمانی شتر با دیگر جانوران اختلاف زیادی دارد، و این اختلاف انگیزه شده که شتر در درازا روزهای سال در بیابان زندگی کند و از بوته‌ها و درختچه‌های گوناگون صحرایی و کویری و حتی از بوته‌های شور و خاردار تغذیه کند. عرب‌ها از زمان‌های بسیار دور از شتر استفاده کرده و می‌کنند. آن‌ها به این حیوان اهلی لقب کشتی صحرا (به عربی: سفینةالصحراء) داده‌اند.",
    """حسین میرزایی می‌گوید مرحله اول پرداخت وام حمایتی کرونا به همگی خانوارهای یارانه‌بگیر متقاضی تکمیل شده است و حال چهار میلیون خانوار که به عنوان "اقشار خاص" و "آسیب‌پذیر" شناسایی شدند، می‌توانند برای یک میلیون تومان وام دیگر درخواست بدهند. آقای میرزایی گفته خانوارهای "آسیب‌پذیر" که شرایط گرفتن وام یک میلیونی اضافی را دارند با پیامک از این امکان مطلع شده‌اند. بنا به گزارش‌های رسمی با شیوع کرونا در ایران یک میلیون نفر بیکار شده‌اند و درآمد کارکنان مشاغل غیررسمی نیز ضربه قابل توجهی خورده است. ارزش ریال هم در هفته‌های اخیر در برابر ارزهای خارجی سقوط کرده است. اقتصاد ایران پیش از شیوع کرونا نیز با مشکلات مزمن رکود، تورم، تحریم و فساد روبرو بود.""",
    "در ۲۲ ژوئن ۱۹۴۱ نیروهای محور در عملیات بارباروسا حمله سنگینی به اتحاد شوروی کرده و یکی از بزرگترین نبردهای زمینی تاریخ بشر را رقم زدند. همچنین جبهه شرقی باعث به دام افتادن نیروهای محور شد و بیش از همه ارتش آلمان نازی را درگیر جنگ فرسایشی کرد. در دسامبر ۱۹۴۱ ژاپن یک در عملیاتی ناگهانی با نام نبرد پرل هاربر به پایگاه دریایی ایالات متحده آمریکا حمله کرد. به دنبال این اتفاق آمریکا نیز بلافاصله علیه ژاپن اعلان جنگ کرد که با حمایت بریتانیا همراه شد. پس از آن متحدین (نیروهای محور در اروپا) نیز با اتحاد ژاپن علیه آمریکا اعلام جنگ کردند. دست‌آوردهای ژاپن در یورش به آمریکا باعث ایجاد این احساس در آسیا شد که آسیا از تسلط غرب خارج شده‌است از این رو بسیاری از ارتش‌های شکست خورده با آنها همراهی کردند."
]
questions = [
    "اشکالی که یک مرکز تقارن دارند",
    "غذای شترچیست؟",
    "وام یارانه به چه کسانی میدهند؟",
    "چرا امریکا وارد جنگ جهانی دوم شد؟"
]
rc_model.mt5_reading_comprehension_inference(context_list, questions, device)

['اگر در یک شکل نقطه ای مانندA وجود داشته باشد که هر',
 'شتر',
 'خانوارهای یارانه بگیر متقاضی',
 'بوسیاری از ارتش های شکست خورده با آنها همراهی کردند']

## ParsiNLU Dataset

In [8]:
!git clone https://github.com/persiannlp/parsinlu
!ls parsinlu
!ls parsinlu/data/reading_comprehension

Cloning into 'parsinlu'...
remote: Enumerating objects: 1434, done.
remote: Counting objects: 100% (182/182), done.
remote: Compressing objects: 100% (98/98), done.
remote: Total 1434 (delta 110), reused 139 (delta 82), pack-reused 1252
Receiving objects: 100% (1434/1434), 27.81 MiB | 19.24 MiB/s, done.
Resolving deltas: 100% (913/913), done.
data  LICENSE  README.md  requirements.txt  scripts  src
dev.json   english_squad_parsinlu_train.json  eval.jsonl  train.jsonl
dev.jsonl  eval.json			      train.json


In [9]:
test_examples = rc_model.load_dataset_test_file(dataset_name="parsinlu", dataset_file="./parsinlu/data/reading_comprehension/eval.json")

100%|██████████| 570/570 [00:00<00:00, 3659.07it/s]


In [10]:
!nvidia-smi
!lscpu

Tue Jul  6 08:06:48 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   63C    P0    31W /  70W |   3520MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [11]:
evaluation_report, evaluation_output = rc_model.mt5_evaluation(test_examples, device, max_length=512, batch_size=64)

#input_data:570
#batch: 9
Start to evaluate test data ...
inference time for step 0: 3.8591785730000083
inference time for step 1: 3.827875364999983
inference time for step 2: 3.8669839110000055
inference time for step 3: 3.8801370929999734
inference time for step 4: 3.9191276210000012
inference time for step 5: 3.980064954999989
inference time for step 6: 4.0397097679999945
inference time for step 7: 4.057680399999981
inference time for step 8: 3.875197791000005
total inference time: 35.30595547699994
total inference time / #samples: 0.06194027276666656
evaluation results:
 OrderedDict([('exact', 27.017543859649123), ('f1', 54.38815682562042), ('total', 570), ('HasAns_exact', 27.017543859649123), ('HasAns_f1', 54.38815682562042), ('HasAns_total', 570), ('best_exact', 27.017543859649123), ('best_exact_thresh', 0.0), ('best_f1', 54.38815682562042), ('best_f1_thresh', 0.0)])


In [12]:
print(evaluation_report)

OrderedDict([('exact', 27.017543859649123), ('f1', 54.38815682562042), ('total', 570), ('HasAns_exact', 27.017543859649123), ('HasAns_f1', 54.38815682562042), ('HasAns_total', 570), ('best_exact', 27.017543859649123), ('best_exact_thresh', 0.0), ('best_f1', 54.38815682562042), ('best_f1_thresh', 0.0)])


In [13]:
for qas_id, context, question, gold_answers, predicted_answer, exact_score, f1_score in evaluation_output[:25]:
  print('{}\t{}\t{}\t{}\t{}\t{}\t{}'.format(qas_id, context, question, gold_answers, predicted_answer, exact_score, f1_score))

9385db85-f5b1-473f-ab2d-d4049f2dbe7d	آب و هوای مناسب برای رشد گندم شرایط ایده‌آل برای رشد گندم ، آب و هوای خنک در دوره رشد رویشی ، آب و هوای معتدل در دوران تشکیل دانه و آب و هوای گرم و خشک در زمان برداشت محصول می‌باشد. بنابراین در مناطقی که زمستانهای سخت دارند، کشت گندم با مشکلاتی از قبیل سرما‌زدگی زمستانی مواجه می‌شود. البته باید بدانیم که گندم در برابر خشکی مقاومت چندانی ندارد و نمی‌تواند به مدت طولانی ، خشکی و کم آبی را تحمل نماید. اما قادر است خود را با شرایط خشک تا حدی تطبیق داده و با تشکیل یاخته‌های کوچکتر که در نهایت سبب تشکیل برگهای کوچک شده و در نتیجه روزنه‌ها کوچکتر می‌شود، سطح تعریق را کاهش دهد و از اثرات سوء کم آبی تا حدی محفوظ بماند.	چه آب و هوایی برای رشد گندم خوب است؟	['آب و هوای خنک در دوره رشد رویشی ، آب و هوای معتدل در دوران تشکیل دانه و آب و هوای گرم و خشک در زمان برداشت محصول']	آب و هوای خنک در دوره رشد رویشی ، آب و هوای	0	0.6
9e5b0592-9a4a-48e4-b090-17ddf5786743	آتش از دیرباز مورد پذیرش ایرانیان باستان و هندوها بوده‌است. هندوها آتش مقدس را آگنی می‌گفتند و برای آن ق

In [14]:
output_file_name = "reading_comprehension-parsinlu_{}_outputs.txt".format(model_name.replace('/','-'))
with open(output_file_name, "w", encoding='utf8') as output_file:
  for qas_id, context, question, gold_answers, predicted_answer, exact_score, f1_score in evaluation_output:
    output_file.write('{}\t{}\t{}\t{}\t{}\t{}\t{}\n'.format(qas_id, context, question, gold_answers, predicted_answer, exact_score, f1_score))
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
upload = drive.CreateFile({'title': output_file_name})
upload.SetContentFile(output_file_name)
upload.Upload()